In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

# Add the action_labeler package to Python path
sys.path.append("../")

In [3]:
from ultralytics import YOLO
import numpy as np
from pathlib import Path

from action_labeler.helpers.detections_helpers import (
    ultralytics_labels_to_xywh,
    xywhs_to_xyxys,
    xyxys_to_xywhs,
    xyxys_to_masks,
)

from action_labeler.helpers.yolov8_dataset import get_data_yaml

In [4]:
from shutil import rmtree

rmtree("./runs/detect", ignore_errors=True)

In [5]:
model = YOLO("yolo12n.pt")

results = model.predict(
    "./samples/dog_laying_down_1.jpg",
    save_txt=True,
)


image 1/1 /home/justin/machine_learning/yolo/action_labeler/notebooks/samples/dog_laying_down_1.jpg: 448x640 2 dogs, 36.1ms
Speed: 3.8ms preprocess, 36.1ms inference, 113.8ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


In [6]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

In [7]:
xyxys = xywhs_to_xyxys(results[0].boxes.xywhn, results[0].orig_shape[::-1])
expected_xyxys = results[0].boxes.xyxy.tolist()

np.testing.assert_allclose(
    np.array(xyxys),
    np.array(expected_xyxys),
    rtol=1e-5,
    atol=1e-5,
    err_msg=f"xyxys and expected_xyxys not close enough",
)

In [8]:
xywhs = xyxys_to_xywhs(results[0].boxes.xyxy, results[0].orig_shape[::-1])
expected_xywhs = results[0].boxes.xywhn.tolist()

np.testing.assert_allclose(
    np.array(xywhs),
    np.array(expected_xywhs),
    rtol=1e-5,
    atol=1e-5,
    err_msg=f"xywhs and expected_xywhs not close enough",
)

In [9]:
from pathlib import Path

labels_path = Path("./runs/detect/predict/labels")

xywhs = []

for label_path in labels_path.glob("*.txt"):
    xywhs.append(ultralytics_labels_to_xywh(label_path))

xywhs = np.array(xywhs)[0]  # ignore batch dimension
expected_xywhs = results[0].boxes.xywhn.tolist()

np.testing.assert_allclose(
    xywhs,
    expected_xywhs,
    rtol=1e-5,
    atol=1e-5,
    err_msg=f"xywhs and expected_xywhs not close enough",
)


np.testing.assert_allclose(
    xywhs_to_xyxys(xywhs, results[0].orig_shape[::-1]),
    results[0].boxes.xyxy.tolist(),
    rtol=1e-4,
    atol=1e-4,
    err_msg=f"xyxys and expected_xyxys not close enough",
)

In [10]:
# masks = xyxys_to_masks(results[0].boxes.xyxy, results[0].orig_shape[::-1])
# masks = np.array(masks)

# # Ensure each mask is the same size as image
# # Ignore batch dimension
# assert (
#     masks.shape[1:] == results[0].orig_shape[::-1]
# ), f"Masks shape {masks.shape[1:]} does not match image shape {results[0].orig_shape[::-1]}"

# # Total number of Trues equals height * width of xyxys
# for mask, xyxy in zip(masks, results[0].boxes.xyxy):
#     total_trues = np.sum(mask)
#     expected_trues = (xyxy[2] - xyxy[0]) * (xyxy[3] - xyxy[1])
#     assert (
#         total_trues == expected_trues
#     ), f"Total number of Trues {total_trues} does not match expected {expected_trues}"

In [11]:
get_data_yaml("./samples", verbose=True)

Dataset path: samples
Class Names:
0: laying down
1: moving
2: sitting
3: standing still
Number of classes: 4


{'names': ['laying down', 'moving', 'sitting', 'standing still'],
 'path': 'dog_dataset_balanced',
 'train': 'train/images',
 'val': 'valid/images'}

In [12]:
from action_labeler.helpers.detections_helpers import Detection

d = Detection.from_ultralytics(results[0])

np.testing.assert_allclose(
    d.image_size,
    results[0].orig_shape[::-1],
    rtol=1e-5,
    atol=1e-5,
    err_msg=f"image_size and expected_image_size not close enough",
)

np.testing.assert_allclose(
    d.xyxy,
    results[0].boxes.xyxy.cpu().numpy(),
    rtol=1e-5,
    atol=1e-5,
    err_msg=f"xyxy and expected_xyxy not close enough",
)

np.testing.assert_allclose(
    d.xywhn,
    results[0].boxes.xywhn.cpu().numpy(),
    rtol=1e-5,
    atol=1e-5,
    err_msg=f"xywhn and expected_xywhn not close enough",
)